# How to plot one large MOC (union) per service

## Import required modules

In [45]:
from elasticsearch import Elasticsearch
from mocpy import MOC

## Connect to the ElasticSearch server

In [ ]:
es = Elasticsearch('http://voparis-elasticsearch.obspm.fr:9200')
indexName = 'moc-index'
target = "mars"

## Get the list of services

In [ ]:
query = {
    "bool": {
         "must": [
             {
                "exists" : {
                    "field" : "coverage"
                }
             },
             {
                "term" : {
                    "target_name.keyword" : target
                }
             }
        ]
    }
}

aggs = {
    "agg-service" : {
        "terms": {
            "field" : "service_title.keyword",
            "size": 100
        },
    }
}


page = es.search(
    index=indexName,
    query=query,
    size=0,
    fields=["*"],
    aggs = aggs
)

services = []
for document in page["hits"]["hits"] :
    data.append(document["_source"])
    print(document["_source"]["service_title"])
    
for bucket in page["aggregations"]["agg-service"]["buckets"] :
    print(bucket["key"])
    services.append(bucket["key"])

## For each service, merge all of its MOCs into one

In [75]:
def mergeMocs(mocsList) :
    mocUnion = MOC.new_empty(1)
    for item in mocsList :
        moc = MOC.from_str(item["fields"]["coverage"][0])
        mocUnion.union(moc)
    return mocUnion

In [77]:
data = []
for service in services :
    print("Merging MOCs of ",service,"...")
    query = {
        "bool": {
             "must": [
                 {
                    "exists" : {
                        "field" : "coverage"
                    }
                },
                {
                    "term" : {
                         "service_title.keyword" : service
                     } 
                 } 
            ]
        }
    }

    page = es.search(
        index=indexName,
        query=query,
        size=1000,
        scroll="2m",
        fields=["coverage"],
        aggs = aggs
    )
    
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']["value"]
    
    mocUnion = MOC.new_empty(1)
    
    progress = 0
    
    while (scroll_size > 0):
        print(progress,"/",page['hits']['total']["value"])
        page = es.scroll(scroll_id = sid, scroll = '2m')
        # Update the scroll ID
        sid = page['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])

        mocUnion.union(mergeMocs(page["hits"]["hits"]))
        progress = progress + scroll_size
        
    data.append({"service":service,"coverage":mocUnion})

Merging MOCs of  hfc1ar_majtrim ...
0 / 1187189
1000 / 1187189
2000 / 1187189
3000 / 1187189
4000 / 1187189
5000 / 1187189
6000 / 1187189
7000 / 1187189
8000 / 1187189
9000 / 1187189
10000 / 1187189
11000 / 1187189
12000 / 1187189
13000 / 1187189
14000 / 1187189
15000 / 1187189
16000 / 1187189
17000 / 1187189
18000 / 1187189


OSError: Parse error: 0: in Digit, got empty input

1: in Many1, got empty input



In [78]:
# TODO : There is an error above, the format of the MOC here seems to be incorrect